## 임베딩 실습 1
### 활용 데이터: hw_sample.csv

### 실습 개요
이 실습에서는 주어진 csv 파일에 포함된 텍스트 데이터에 대해 Okt 형태소 분석기를 사용하여 토큰화를 수행하고, 그 결과를 기반으로 Word2Vec 모델 학습하는 실습입니다. KoNLPy 라이브러리 Okt 형태소 분석기를 활용하여 토큰화를 수행하고, 토큰화된 데이터를 활용해 Word2Vec모델을 학습한 다음 저장합니다.


### 실습 절차
1. 라이브러리 설치 및 임포트: 필요한 라이브러리 (gensim, konlpy)를 설치하고 임포트합니다.
2. csv 데이터를 pandas를 활용해 읽습니다.
3. 데이터 프레임 내에서 sentence1에 대해 Okt 형태소 분석기를 활용하여 토큰화를 수행합니다.
4. 토큰화된 데이터를 활용해 Word2Vec 모델 학습합니다.
5. Word2Vec 모델을 저장합니다.
6. 데이터 프레임 내에서 sentence1, sentence2칼럼에 대해 문장을 벡터로 변환합니다. (이때, Okt 형태소 분석기를 활용하여 토큰화를 수행합니다.)
8. 코사인 유사도를 계산합니다.
9. 계산된 결과값을 새로운 컬럼 (cosine)을 생성하여 저장합니다.
10. 결과 파일을 result.csv로 저장합니다.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install konlpy gensim

In [21]:
import json
import pandas as pd

from gensim.models import Word2Vec
from konlpy.tag import Okt
import numpy as np
from scipy.spatial.distance import cosine


In [7]:
df = pd.read_csv('/content/drive/MyDrive/NLP 실습/1 11/hw_sample.csv')

In [8]:
df.head(2)

,sentence1,sentence2
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ..."
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어..."


In [9]:
sample_sentences1 = df['sentence1']

In [10]:
# 토큰화
okt = Okt()
sentences1_tokenized = [okt.morphs(sentence) for sentence in sample_sentences1]

In [11]:
# Word2Vec 모델 학습
model = Word2Vec(sentences1_tokenized, vector_size=100, window=5, min_count=1, workers=4)

In [15]:
# 문장을 벡터로 변환하는 함수
def sentence_vector(sentence, model):
  # 토큰화
  words = okt.morphs(sentence)

  # 단어 벡터화
  # 단어가 모델에 있을 경우 벡터를 추출
  word_vectors = [model.wv[word] for word in words if word in model.wv]

  # 벡터의 유사도를 구하기 위해 각 단어마다 나온 값들에 대한 평균을 계산
  if not word_vectors:
      return np.zeros(model.vector_size)

  return np.mean(word_vectors, axis=0)

In [18]:
df['sentence1_vec'] = df['sentence1'].apply(lambda x: sentence_vector(x, model))

In [19]:
df['sentence2_vec'] = df['sentence2'].apply(lambda x: sentence_vector(x, model))

In [26]:
df['cosine_similarity'] = df.apply(lambda x: 1 - cosine(x['sentence1_vec'], x['sentence2_vec']), axis=1)

In [25]:
df.to_csv('result.csv', index=False)